In [ ]:
import numpy as np # linear algebra
from numpy import random
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import re
import logging        
import gensim
import xgboost as xgb
from tqdm.notebook import tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

df = pd.read_csv('/kaggle/input/train-cleaned/trainset_cleaned.csv')
train = df

-------
* Using **oversampling** to avoid information lose!
* first train test split, second oversampling!

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
train.groupby('sentiment').clean_content.count().plot.bar(ylim=0)
#plt.show()

## label data 
#from sklearn import preprocessing
#le = preprocessing.LabelEncoder()
#train['sentiment_L'] = le.fit_transform(train['sentiment'])
#train.head(3)

# train-test split 
X = train[['clean_content','sentiment']]
y = train.sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)



# oversampling on training set
import imblearn
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
#X_ros,y_ros  = ros.fit_sample(X, y)
X_ros,y_ros  = ros.fit_sample(X, y)

print(X_ros.shape[0] - X_train.shape[0], 'new random picked points')

X_ros.index.name = 'id'
#y_ros.index.name = 'id'
#train_ros = pd.merge(X_ros,
#                    y_ros,
#                    how = 'left',
#                    on = 'id')
print('Train set shape after oversampling: ', X_ros.shape )

# plot balanced
fig = plt.figure(figsize=(8,6))
X_ros.groupby('sentiment').clean_content.count().plot.bar(ylim=0)
plt.show()

%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

test = pd.read_csv('/kaggle/input/topic907/topics.csv')

##Logistic regression
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_ros.clean_content.values.astype('U'), X_ros.sentiment.values.astype('U'))

y_pred = logreg.predict(test['clean_content'][100000:200000].values.astype('U'))
print('Logistic regression')
#print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred))


In [ ]:
y_pred = logreg.predict(test['clean_content'][:40000].values.astype('U'))

In [ ]:
len(test['clean_content'][:40000])

In [ ]:
from pandas import DataFrame
y_pred = pd.DataFrame(y_pred, columns = ['sentiment'] )
y_pred.to_csv('sapred40000.csv', index = False)

------

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
train.groupby('sentiment').clean_content.count().plot.bar(ylim=0)
plt.show()

In [ ]:
# train-test split 
X = train[['clean_content','sentiment']]
y = train.sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

# oversampling on training set
import imblearn
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X_ros,y_ros  = ros.fit_sample(X, y)

print(X_ros.shape[0] - X_train.shape[0], 'new random picked points')
X_ros.index.name = 'id'
print('Train set shape after oversampling: ', X_ros.shape )

# plot balanced
fig = plt.figure(figsize=(8,6))
X_ros.groupby('sentiment').clean_content.count().plot.bar(ylim=0)
plt.show()

---
## supervised learning

* step1: feature extraction [sklearn.feature_extraction.text ](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text)
The goal of using **tf-idf** instead of the raw frequencies of occurrence of a token in a given document is to **scale down** the impact of tokens that occur very frequently in a given corpus and that are hence empirically less informative than features that occur in a small fraction of the training corpus. 

In [ ]:
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

X_train = X_ros.clean_content
y_train = X_ros.sentiment
X_test = X_test.clean_content
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)


In [ ]:

##Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB())])
nb.fit(X_train.values.astype('U'), y_train)
y_pred = nb.predict(X_test.values.astype('U'))
print('Naive Bayes classifier')
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

##Linear SVM
from sklearn.linear_model import SGDClassifier
sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train.values.astype('U'), y_train)

y_pred = sgd.predict(X_test.values.astype('U'))
print('Liear SVM')
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

##Logistic regression
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train.values.astype('U'), y_train)

y_pred = logreg.predict(X_test.values.astype('U'))
print('Logistic regression')
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

## LinearSVC
from sklearn.svm import LinearSVC
lsvc = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LinearSVC()),
               ])
lsvc.fit(X_train.values.astype('U'), y_train)
y_pred = lsvc.predict(X_test.values.astype('U'))
print('LinearSVC')
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

## RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(max_depth=2, random_state = 42)),
               ])
rf.fit(X_train.values.astype('U'), y_train)
y_pred = lsvc.predict(X_test.values.astype('U'))
print('RandomForestClassifier')
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))


In [ ]:
test = pd.read_csv('/kaggle/input/topic907/topics.csv')

In [ ]:
##Logistic regression
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_ros.clean_content.values.astype('U'), X_ros.sentiment.values.astype('U'))

y_pred = logreg.predict(test['clean_content'][0:40000].values.astype('U'))
print('Logistic regression')
#print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred))


In [ ]:
test['clean_content'][40000:]

In [ ]:
test = pd.read_csv('/kaggle/input/topic907/topics.csv')


In [ ]:
y_pred = logreg.predict(test['clean_content'][40000:].values.astype('U'))

In [ ]:
len(y_pred)

In [ ]:
y_pred.to_csv('sapred40000.csv', index = False)

In [ ]:
len(X)

In [ ]:
len(X_train)

In [ ]:
y_pred = logreg.predict(test.clean_content)
test['sentiment'] = y_pred
test.to_csv('sentiment_test_results.csv', index = False)

In [ ]:
from sklearn.model_selection import cross_val_score
models = [nb, sgd, logreg, lsvc, rf]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, X_train, y_train, cv=CV, scoring='accuracy')
    #accuracies = accuracy_score(y_pred, y_test)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
    
import seaborn as sns

sns.boxplot(x='fold_idx', y='accuracy', data=cv_df)
sns.stripplot(x='fold_idx', y='accuracy', data=cv_df, 
              size=16, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

cv_df.groupby('fold_idx').accuracy.mean()

----

In [ ]:
train['category_id'] = train['sentiment'].factorize()[0]
from io import StringIO
category_id_df = train[['sentiment', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'sentiment']].values)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(train.clean_content).toarray()
labels = train.category_id
print(features.shape)

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(8,6))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels = category_id_df.sentiment.values, yticklabels=category_id_df.sentiment.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

Since feature and feature1's shape are the same so there is no duplications so we we can use the same train_text_split as before. 

In [ ]:
#from sklearn.feature_selection import chi2
#import numpy as np

#N = 2
#for Product, category_id in sorted(category_to_id.items()):
#    features_chi2 = chi2(features, labels == category_id)
#    indices = np.argsort(features_chi2[0])
#    feature_names = np.array(tfidf.get_feature_names())[indices]
#    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
#    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
#    print("# '{}':".format(Product))
#    print("  . Most correlated unigrams:\n       . {}".format('\n       . '.join(unigrams[-N:])))
#    print("  . Most correlated bigrams:\n       . {}".format('\n       . '.join(bigrams[-N:])))

In [ ]:
from IPython.display import display

for predicted in category_id_df.category_id:
    for actual in category_id_df.category_id:
        if predicted != actual and conf_mat[actual, predicted] >= 6:
            print("'{}' predicted as '{}' : {} examples.".format(id_to_category[actual], id_to_category[predicted], conf_mat[actual, predicted]))
            display(train2.loc[indices_test[(y_test == actual) & (y_pred == predicted)]][['sentiment', 'text']])
            print('')

In [ ]:
model.fit(features, labels)

In [ ]:
from sklearn.feature_selection import chi2

N = 2
for Product, category_id in sorted(category_to_id.items()):
    indices = np.argsort(model.coef_[category_id])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 1][:N]
    bigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 2][:N]
    print("# '{}':".format(Product))
    print("  . Top unigrams:\n       . {}".format('\n       . '.join(unigrams)))
    print("  . Top bigrams:\n       . {}".format('\n       . '.join(bigrams)))

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred, 
                                    target_names=train2['sentiment'].unique()))

----------------
Word2vec embedding and Logistic Regression

Doc2vec and Logistic Regression

-- Taking the linear combination of every term in the document creates a random walk with bias process in the word2vec space.

------------
# LSTM

In [ ]:
!pip install tokenizer

In [ ]:
!pip install --upgrade pip
!pip install tweet-preprocessor 2>/dev/null 1>/dev/null

!pip install transformers
!pip install tensorboardx
!pip install simpletransformers

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

import re

from nltk.corpus import stopwords
from nltk import word_tokenize

import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

from IPython.core.interactiveshell import InteractiveShell

import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot
import tokenizer
InteractiveShell.ast_node_interactivity = 'all'

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split

from kaggle_datasets import KaggleDatasets

import transformers
from transformers import TFAutoModel, AutoTokenizer

STOPWORDS = set(stopwords.words('english'))

In [ ]:
## label data 
#from sklearn import preprocessing
#le = preprocessing.LabelEncoder()
#train['sentiment_L'] = le.fit_transform(train['sentiment'])
#train.head(3)

# train-test split 
X = train[['clean_content','sentiment']]
y = train.sentiment
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)



# oversampling on training set
import imblearn
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
#X_ros,y_ros  = ros.fit_sample(X, y)
X_ros,y_ros  = ros.fit_sample(X, y)

#print(X_ros.shape[0] - X_train.shape[0], 'new random picked points')

X_ros.index.name = 'id'
#y_ros.index.name = 'id'
#train_ros = pd.merge(X_ros,
#                    y_ros,
#                    how = 'left',
#                    on = 'id')
print('Train set shape after oversampling: ', X_ros.shape )

# plot balanced
fig = plt.figure(figsize=(8,6))
X_ros.groupby('sentiment').clean_content.count().plot.bar(ylim=0)
plt.show()

In [ ]:
# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
#print(t.word_docs)


In [ ]:
from keras.preprocessing.text import Tokenizer
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(X_ros['clean_content'].values.astype(str))

#X_ros = df[['clean_content','sentiment']]
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100


X = t.texts_to_sequences(X_ros['clean_content'].values.astype(str))
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Y = pd.get_dummies(X_ros['sentiment']).values
print('Shape of label tensor:', Y.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(13, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

epochs = 8
batch_size = 64

history = model.fit(X, Y, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

In [ ]:
Y = pd.get_dummies(X_ros['sentiment']).values
print('Shape of label tensor:', Y.shape)

In [ ]:
len(Y)

In [ ]:
# save model to single file
model.save('lstm_model.h1')

In [ ]:
#X_pred = t.texts_to_sequences(test['clean_content'].values.astype(str))
#X_pred = pad_sequences(X_pred, maxlen=MAX_SEQUENCE_LENGTH)
#y_predict = model.predict_classes(X_pred)
#test['sentiments'] = y_predict

In [ ]:
test = pd.read_csv('/kaggle/input/topic907/topics.csv')
test

In [ ]:
test['clean_content'][:10000]

In [ ]:
X_test = t.texts_to_sequences(test['clean_content'][:10000].values.astype(str))
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
yhat10000 = model.predict_classes(X_test)
from pandas import DataFrame
yhat10000 = pd.DataFrame(yhat10000, columns = ['sentiment'] )
yhat10000.to_csv('sapred_10000.csv', index = False)

In [ ]:
test['clean_content'][10000:20000]

In [ ]:
X_test = t.texts_to_sequences(test['clean_content'][10000:20000].values.astype(str))
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
yhat20000 = model.predict_classes(X_test)
from pandas import DataFrame
yhat20000 = pd.DataFrame(yhat20000, columns = ['sentiment'] )
yhat20000.to_csv('sapred_20000.csv', index = False)

In [ ]:
test['clean_content'][20000:30000]

In [ ]:
X_test = t.texts_to_sequences(test['clean_content'][20000:30000].values.astype(str))
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
yhat30000 = model.predict_classes(X_test)
yhat30000 = pd.DataFrame(yhat30000, columns = ['sentiment'] )
yhat30000.to_csv('sapred_30000.csv', index = False)

In [ ]:
test['clean_content'][30000:40000]

In [ ]:
X_test = t.texts_to_sequences(test['clean_content'][30000:40000].values.astype(str))
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
yhat40000 = model.predict_classes(X_test)
from pandas import DataFrame
yhat40000 = pd.DataFrame(yhat40000, columns = ['sentiment'] )
yhat40000.to_csv('sapred_40000.csv', index = False)

In [ ]:
test = pd.read_csv('/kaggle/input/topic907/topics.csv')
test
test['clean_content'][40000:]

In [ ]:
X_test = t.texts_to_sequences(test['clean_content'][40000:].values.astype(str))
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
yhat50000 = model.predict_classes(X_test)
yhat50000 = pd.DataFrame(yhat50000, columns = ['sentiment'] )
yhat50000.to_csv('sapred_50000.csv', index = False)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
MAX_SEQUENCE_LENGTH = 250
from keras.preprocessing.text import Tokenizer
# create the tokenizer
t = Tokenizer()
from keras.models import load_model
# load model from single file
model = load_model('lstm_model.h1')
# make predictions
X_test = t.texts_to_sequences(test['clean_content'][10000:20000].values.astype(str))
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
yhat = model.predict_classes(X_test)
#target['sent_LSTM'] = yhat
#target.sentiment = le.inverse_transform(target.sentlabel)

In [ ]:
X_test

In [ ]:
yhat.to_cav('sa10000.csv', index = False)

In [ ]:
#making predictions using previous models
ylr = logreg.predict(target.samples)
yLsvm = sgd.predict(target.samples)
 ynb = nb.predict(target.samples)
ylr = logreg.predict(target.samples)
yLsvm = sgd.predict(target.samples)
ynb = nb.predict(target.samples)
ylr = logreg.predict(target.samples)
target['sent_logreg'] = ylr
target['sent_Lsvm'] = yLsvm
target['sent_Naive_Bayes'] = ynb
target.rename(columns={"sentiment": "sent_lstm"})
target = target.drop(['confident_score','sentlabel'], axis=1)
target.loc['accuracy'] = ['', '0.880', '0.8836597005101201','0.7806483462234656', '0.8025341451374033']
yLsvc = lsvc.predict(target.samples)
target['sent_logreg'] = yLsvc
target.to_csv('sample_results.csv',index = False)

In [ ]:
import types
import tempfile
import keras.models

def make_keras_picklable():
    def __getstate__(self):
        model_str = ""
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            keras.models.save_model(self, fd.name, overwrite=True)
            model_str = fd.read()
        d = { 'model_str': model_str }
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            model = keras.models.load_model(fd.name)
        self.__dict__ = model.__dict__


    cls = keras.models.Model
    cls.__getstate__ = __getstate__
    cls.__setstate__ = __setstate__

In [ ]:
import keras
import pickle

make_keras_picklable()

m = keras.models.Sequential()
m.add(keras.layers.Dense(10, input_shape=(10,)))
m.compile(optimizer='sgd', loss='mse')

pickle.dumps(m)


In [ ]:
# load model 
make_keras_picklable()

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))